<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Константная-модель" data-toc-modified-id="Константная-модель-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Константная модель</a></span><ul class="toc-item"><li><span><a href="#Проверка-лучшей-модели" data-toc-modified-id="Проверка-лучшей-модели-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Проверка лучшей модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**План по выполнению проекта**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Вывод.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорт библиотек

In [3]:
import pandas as pd
import numpy as np
import re
import torch
import transformers

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

from tqdm import notebook
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Подготовка

In [4]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
data.sample(5)

,Unnamed: 0,text,toxic
129660,129793,"""Thank you for experimenting with Wikipedia. ...",0
73497,73569,ou leftist Wikipedia scum are an insignificant...,1
74835,74911,REDIRECT Talk:1968–69 Mexican Primera División...,0
58152,58216,Sorry Caden. I thank you for what you did on t...,0
130768,130904,are you even remotely bothered about your GA n...,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Данные загружены корректно, первичный осмотр проведен.</font>

In [6]:
#столбец неинформативный, удалим его
del data['Unnamed: 0']

In [7]:
#для удобства приведём текст в нижнийц регистр
data['text'] = data['text'].str.lower()

In [8]:
#проверим наличие дубликатов
data.duplicated().sum()

45

In [9]:
#удалим дубликаты
data = data.drop_duplicates()
data.duplicated().sum()

0

In [10]:
#проверим наличие пропусков
data.isna().sum()

text     0
toxic    0
dtype: int64

In [11]:
#посотрим на баланс классов
data['toxic'].value_counts()

0    143076
1     16171
Name: toxic, dtype: int64

In [12]:
#напишем функцию для токенизации и лемматизации текста

def clear(text):
    tokenized = nltk.word_tokenize(text)
    joined = ' '.join(tokenized)
    clear_text = re.sub(r"[^a-zA-Z' ]", ' ', joined)
    output = ' '.join(clear_text.split())
    return output

#лемматизация
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    
    return lemmatized_output

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
tqdm.pandas() 
data['token_text'] = data['text'].progress_apply(clear)

100%|██████████| 159247/159247 [01:50<00:00, 1442.55it/s]


In [14]:
# tqdm.pandas() 
# data['lemm_text'] = data['token_text'].progress_apply(lemmatize)

In [15]:
data.sample(5)

,text,toxic,token_text
56889,quick questions \nhello. is your name now or h...,0,quick questions hello is your name now or has ...
32297,the anti-hal turner rhetoric has surpassed mad...,0,the anti hal turner rhetoric has surpassed mad...
39320,jumping the homeless\ncould cartman jumping th...,0,jumping the homeless could cartman jumping the...
9008,good edit \n\nhi rw - good edit to fr. i thoug...,0,good edit hi rw good edit to fr i thought that...
91274,the whole article is only from the liberal bib...,0,the whole article is only from the liberal bib...


**Вывод:**

Данные были предобработаны:

- Удален неинформативный столбец
- Очищены от дубликатов
- Столбец text приведен в нижний регистр, токенизирован (разбит на токены: отдельные фразы, слова, символы) и лемматизирован

Также был замечен сильный дисбаланс классов(отличие примерно в 8 раз), поэтому впоследствии попробуем применить техники добаланса.

## Обучение

### LogisticRegression

In [73]:
# выделим обучающие и целевой признаки
target = data['toxic']
features = data['lemm_text'].values

In [16]:
# выделим обучающие и целевой признаки
target = data['toxic']
features = data['token_text'].values

In [17]:
#разобъём датасет на обучающий и тестовый в соотношении 65:35
features_train, features_test, target_train, target_test = train_test_split (
    features, 
    target, 
    test_size=0.35, 
    random_state=12345
)

In [18]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train) 
features_test =  count_tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
LR = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1', C=3)

In [18]:
scores = cross_val_score (
    LR, 
    features_train, 
    target_train, 
    cv=10,
    scoring='f1'
)

final_score = sum(scores) / len(scores)
print(final_score)

0.7732735165115742


In [19]:
# добалансировка добавлением параметра class_weight='balanced'
LR = LogisticRegression(class_weight='balanced', random_state=12345, solver='liblinear', penalty='l1', C=4)

In [20]:
scores = cross_val_score (
    LR, 
    features_train, 
    target_train, 
    cv=10,
    scoring='f1'
)

final_score = sum(scores) / len(scores)
print(final_score)

0.7543286282003089


Значение f1_score удовлетворяет поставленному условию. Значение получилось больше 75 даже без добалансировки.

### BERT

Посмотрим на нейронную сеть от Google - Bert.

Мощности компьютера не хватит на обработку целого датасета, возьмем выборку из 1000 элементов.

In [37]:
data_bert = data.sample(1000).reset_index(drop=True)

In [38]:
#загрузим необходимое
configuration = transformers.DistilBertConfig()
model = transformers.DistilBertModel(configuration)
pretrained_weights = 'distilbert-base-uncased'
tokenizer_class = transformers.DistilBertTokenizer

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [39]:
tokenized = data_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

In [40]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [41]:
len(padded[0])

512

In [42]:
padded.shape, attention_mask.shape

((1000, 512), (1000, 512))

In [43]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [44]:
features = np.concatenate(embeddings)
target = data_bert['toxic']

In [45]:
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(
    features,
    target,
    test_size = 0.5,
    random_state = 12345
)

In [56]:
LR = LogisticRegression()

grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

grid = GridSearchCV(logreg, grid, cv=10)
grid.fit(features_train_bert,target_train_bert)

grid.best_params_

{'C': 0.001, 'penalty': 'l2'}

In [64]:
LR = LogisticRegression()

scores = cross_val_score (
    LR, 
    features_train_bert, 
    target_train_bert, 
    cv=10,
    scoring='f1'
)

final_score = sum(scores) / len(scores)
print(final_score)

0.05357142857142858


Значение метрики получилось намного меньше порога, но нужно учитывать, что мы использовали данные не с всего датасета, а также ограничивались максимальной длиной в 512 уникальных значений.

Эти ограничения могли сильно исказить результат.

## Константная модель

В целом, нам надо предсказывать токсичные комментарии, тогда если у нас будет модель предсказывающая только их, то её f1_score будет равен:

In [68]:
dummy_1 = DummyClassifier(random_state=12345, strategy='constant', constant=1)

In [69]:
dummy_1.fit(features_train, target_train)
predictions = dummy_1.predict(features_test)

f1_score(terget_test, predictions)

0.1812041116005874

Что в разы меньше скора полученного с нашей модели.

### Проверка лучшей модели

In [78]:
LR = LogisticRegression(random_state=12345, solver='liblinear', penalty='l1', C=3).fit(features_train, target_train)

print('Значение f1_score:',f1_score(terget_test, LR.predict(features_test)))

Значение f1_score: 0.7770083102493075


## Выводы

Необходимо было создать инструмент, который будет искать токсичные комментарии.

Для начала был загружены и подготовлены данные:

- убрали неинформативные столбцы
- удалили дубликаты 
- написали функции для токенизации и лемматизации текста
также на этаппе подготовки был замечен сильный дисбаланс классов

Далее была обучена логистическая регрессия на всех данных, <br>
получили значение метрики **f1_score: 0.7817056121429997**

Далее протестировали модель - нейрнонную сеть Bert,<br>
Так как модель ресурсозатратная, обучение происходила на небольшой выборке данных,<br>
А также пришлось сократить уникальные значения текстов до 512 значений,<br>
Что скорее всего сильно исказило результаты, <br>
получено значение **f1_score: 0.1411764705882353**

Лучшая модель - логистическая регрессия, результат в разы лучше, чем результат константной модели.

Результат на тестовой выборке: **0.7770083102493075**